In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

data_path = "/content/drive/MyDrive/FYP_dataset/finetune_file.jsonl"

ds = load_dataset("json", data_files=data_path, split="train")

# Limit to first 4000 rows
ds = ds.select(range(min(4000, len(ds))))

print(ds)
print("Columns:", ds.column_names)
print("Sample row:\n", ds[0])


Dataset({
    features: ['id', 'task', 'instruction', 'input', 'output'],
    num_rows: 4000
})
Columns: ['id', 'task', 'instruction', 'input', 'output']
Sample row:
 {'id': '00-2001_S::0000', 'task': 'simplify_summary', 'instruction': 'පහත නීතිමය පෙළ සරල සිංහලෙන් සාරාංශ කරන්න. (අදාළ වන්නේ කාටද, මොනවා කරන්නද කියලා පැහැදිලි කරන්න.)', 'input': 'ඳහහන්වන ආණ්ඩුක්\u200dරම ව්\u200dයවස්ථා සංශාෝධනය\n[සහතිකය සටහන්\u200c කළේ 2001 ඔක්තෝබර්\u200c මස 03 වන දිත]\nආණ්ඩුවේ නියමය පරිඳි මුදුණය කරන ලදී.', 'output': '{"plain_summary": "මෙම නීතිමය කොටස සරල සිංහලෙන් තේරුම් ගත හැකි ලෙස සාරාංශ කර ඇත.", "applies_to": ["(පෙළෙන් පැහැදිලි නැත)"], "main_points": ["මෙම කොටස නීතිමය විධිවිධාන හෝ අර්ථ දැක්වීම් අඩංගු විය හැක."], "actions_for_non_expert": ["ඔබට අදාළදැයි තහවුරු කර ගැනීමට සම්පූර්ණ ලේඛනයේ සම්බන්ධ කොටස්ද පරීක්ෂා කරන්න."], "risks_or_warnings": ["මෙම කොටසෙන් පමණක් දඩ/දඬුවම් පැහැදිලි නොවිය හැක."], "detected": {"has_obligation": false, "has_deadline": false, "has_penalty": false, "has_prohibition": false}, "sour

In [ ]:
!pip -q install -U transformers datasets accelerate peft trl bitsandbytes sentencepiece


In [ ]:
import trl, inspect
from trl import SFTTrainer
print("TRL version:", trl.__version__)
print("SFTTrainer signature:\n", inspect.signature(SFTTrainer.__init__))


TRL version: 0.27.1
SFTTrainer signature:
 (self, model: 'str | PreTrainedModel | PeftModel', args: trl.trainer.sft_config.SFTConfig | transformers.training_args.TrainingArguments | None = None, data_collator: collections.abc.Callable[[list[typing.Any]], dict[str, typing.Any]] | None = None, train_dataset: datasets.arrow_dataset.Dataset | datasets.iterable_dataset.IterableDataset | None = None, eval_dataset: datasets.arrow_dataset.Dataset | datasets.iterable_dataset.IterableDataset | dict[str, datasets.arrow_dataset.Dataset | datasets.iterable_dataset.IterableDataset] | None = None, processing_class: transformers.tokenization_utils_base.PreTrainedTokenizerBase | transformers.processing_utils.ProcessorMixin | None = None, compute_loss_func: collections.abc.Callable | None = None, compute_metrics: collections.abc.Callable[[transformers.trainer_utils.EvalPrediction], dict] | None = None, callbacks: list[transformers.trainer_callback.TrainerCallback] | None = None, optimizers: tuple[torch.

In [ ]:
from datasets import load_dataset

data_path = "/content/drive/MyDrive/FYP_dataset/finetune_file.jsonl"

ds = load_dataset("json", data_files=data_path, split="train")
ds = ds.select(range(min(4000, len(ds))))

print(ds)
print("Columns:", ds.column_names)

print("Sample row:\n", ds[0])

possible_input = ["input", "document", "doc", "text", "source", "article", "content"]
possible_target = ["output", "summary", "target", "answer", "completion", "response", "label"]

input_col = next((c for c in possible_input if c in ds.column_names), None)
target_col = next((c for c in possible_target if c in ds.column_names), None)

print("Auto-detected input_col:", input_col)
print("Auto-detected target_col:", target_col)

assert input_col is not None, "Couldn't detect input column. Set input_col manually."
assert target_col is not None, "Couldn't detect target column. Set target_col manually."


Dataset({
    features: ['id', 'task', 'instruction', 'input', 'output'],
    num_rows: 4000
})
Columns: ['id', 'task', 'instruction', 'input', 'output']
Sample row:
 {'id': '00-2001_S::0000', 'task': 'simplify_summary', 'instruction': 'පහත නීතිමය පෙළ සරල සිංහලෙන් සාරාංශ කරන්න. (අදාළ වන්නේ කාටද, මොනවා කරන්නද කියලා පැහැදිලි කරන්න.)', 'input': 'ඳහහන්වන ආණ්ඩුක්\u200dරම ව්\u200dයවස්ථා සංශාෝධනය\n[සහතිකය සටහන්\u200c කළේ 2001 ඔක්තෝබර්\u200c මස 03 වන දිත]\nආණ්ඩුවේ නියමය පරිඳි මුදුණය කරන ලදී.', 'output': '{"plain_summary": "මෙම නීතිමය කොටස සරල සිංහලෙන් තේරුම් ගත හැකි ලෙස සාරාංශ කර ඇත.", "applies_to": ["(පෙළෙන් පැහැදිලි නැත)"], "main_points": ["මෙම කොටස නීතිමය විධිවිධාන හෝ අර්ථ දැක්වීම් අඩංගු විය හැක."], "actions_for_non_expert": ["ඔබට අදාළදැයි තහවුරු කර ගැනීමට සම්පූර්ණ ලේඛනයේ සම්බන්ධ කොටස්ද පරීක්ෂා කරන්න."], "risks_or_warnings": ["මෙම කොටසෙන් පමණක් දඩ/දඬුවම් පැහැදිලි නොවිය හැක."], "detected": {"has_obligation": false, "has_deadline": false, "has_penalty": false, "has_prohibition": false}, "sour

In [ ]:
SYSTEM = (
    "ඔබ ශ්‍රී ලංකාවේ නීතිමය ලේඛන විශ්ලේෂක AI සහායකයෙක්. "
    "ඔබගේ කාර්යය: සිංහල නීතිමය ලේඛන සාරාංශ කරමින්, නීතිමය විශ්ලේෂණයක් සහ "
    "පරිශීලකයාට තීරණ ගැනීමට උපකාරී ක්‍රියාමාර්ග/උපදෙස් සපයන්න. "
    "නීතිමය උපදෙස් ලෙස නොව, සාමාන්‍ය තොරතුරු ලෙස ඉදිරිපත් කරන්න."
)

def format_example(example):
    doc = example[input_col]
    target = example[target_col]

    prompt = (
        f"<|system|>\n{SYSTEM}\n"
        f"<|user|>\n"
        f"මෙම නීතිමය ලේඛනය කියවා පහත ආකෘතියට අනුව පිළිතුර ලබා දෙන්න:\n\n"
        f"1) සාරාංශය\n2) නීතිමය විශ්ලේෂණය\n3) ක්‍රියාමාර්ග/තීරණ උපදෙස්\n\n"
        f"ලේඛනය:\n{doc}\n"
        f"<|assistant|>\n{target}"
    )
    return {"text": prompt}

ds_formatted = ds.map(format_example, remove_columns=ds.column_names)
print(ds_formatted[0]["text"][:800])


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

<|system|>
ඔබ ශ්‍රී ලංකාවේ නීතිමය ලේඛන විශ්ලේෂක AI සහායකයෙක්. ඔබගේ කාර්යය: සිංහල නීතිමය ලේඛන සාරාංශ කරමින්, නීතිමය විශ්ලේෂණයක් සහ පරිශීලකයාට තීරණ ගැනීමට උපකාරී ක්‍රියාමාර්ග/උපදෙස් සපයන්න. නීතිමය උපදෙස් ලෙස නොව, සාමාන්‍ය තොරතුරු ලෙස ඉදිරිපත් කරන්න.
<|user|>
මෙම නීතිමය ලේඛනය කියවා පහත ආකෘතියට අනුව පිළිතුර ලබා දෙන්න:

1) සාරාංශය
2) නීතිමය විශ්ලේෂණය
3) ක්‍රියාමාර්ග/තීරණ උපදෙස්

ලේඛනය:
ඳහහන්වන ආණ්ඩුක්‍රම ව්‍යවස්ථා සංශාෝධනය
[සහතිකය සටහන්‌ කළේ 2001 ඔක්තෝබර්‌ මස 03 වන දිත]
ආණ්ඩුවේ නියමය පරිඳි මුදුණය කරන ලදී.
<|assistant|>
{"plain_summary": "මෙම නීතිමය කොටස සරල සිංහලෙන් තේරුම් ගත හැකි ලෙස සාරාංශ කර ඇත.", "applies_to": ["(පෙළෙන් පැහැදිලි නැත)"], "main_points": ["මෙම කොටස නීතිමය විධිවිධාන හෝ අර්ථ දැක්වීම් අඩංගු විය හැක."], "actions_for_non_expert": ["ඔබට අදාළදැයි තහවුරු කර ගැනීමට සම්පූර්ණ ලේඛනයේ සම්බ


In [ ]:
# 80% train, 20% temp
split1 = ds_formatted.train_test_split(test_size=0.2, seed=42)
train_ds = split1["train"]
temp_ds  = split1["test"]

# temp 20% -> 10% val, 10% test (split half-half)
split2 = temp_ds.train_test_split(test_size=0.5, seed=42)
val_ds  = split2["train"]
test_ds = split2["test"]

print("Train:", len(train_ds))
print("Val  :", len(val_ds))
print("Test :", len(test_ds))


Train: 3200
Val  : 400
Test : 400


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # small + instruct :contentReference[oaicite:5]{index=5}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,   # ✅ force fp16 weights/params (prevents bf16 grads)
)
model.config.use_cache = False



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

In [ ]:
# Ensure pad token exists (important for batching)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"


In [ ]:
max_seq_length = 1024  # if OOM, reduce to 768 or 512


In [ ]:
sample = train_ds[0]["text"]

enc = tokenizer(
    sample,
    truncation=True,
    max_length=1024,
    padding=False,
    return_tensors=None
)

print("Token count:", len(enc["input_ids"]))
print("First 30 ids:", enc["input_ids"][:30])
print("Last 30 ids:", enc["input_ids"][-30:])

# Decode check (should look like original text)
decoded = tokenizer.decode(enc["input_ids"][:300], skip_special_tokens=False)
print("\nDecoded preview:\n", decoded)


Token count: 1024
First 30 ids: [27, 91, 8948, 91, 397, 150451, 148982, 27982, 115, 223, 48749, 232, 378, 235, 148224, 48749, 241, 27982, 114, 121, 54642, 224, 147677, 48749, 237, 147878, 48749, 248, 27982, 114]
Last 30 ids: [147095, 48749, 237, 150454, 48749, 240, 147677, 148224, 150453, 147646, 48749, 248, 198, 146757, 54642, 115, 48749, 237, 148573, 148981, 48749, 240, 147878, 148224, 147646, 48749, 237, 27982, 115, 226]

Decoded preview:
 <|system|>
ඔබ ශ්‍රී ලංකාවේ නීතිමය ලේඛන විශ්ලේෂක AI සහායකයෙක්. ඔබගේ කාර්යය: සිංහල නීතිමය ලේඛන සාරාංශ කරමින්, නීතිමය විශ්ලේෂණයක් සහ පරිශීලකයාට තීරණ ගැනීමට උපකාරී ක්‍රියාමාර්ග/උපදෙස් සපයන්න. නීතිමය උපදෙස් ලෙස නොව, සාමාන


In [ ]:
print(train_ds.column_names)
print(train_ds[0]["text"][:300])


['text']
<|system|>
ඔබ ශ්‍රී ලංකාවේ නීතිමය ලේඛන විශ්ලේෂක AI සහායකයෙක්. ඔබගේ කාර්යය: සිංහල නීතිමය ලේඛන සාරාංශ කරමින්, නීතිමය විශ්ලේෂණයක් සහ පරිශීලකයාට තීරණ ගැනීමට උපකාරී ක්‍රියාමාර්ග/උපදෙස් සපයන්න. නීතිමය උපදෙස් ලෙස නොව, සාමාන්‍ය තොරතුරු ලෙස ඉදිරිපත් කරන්න.
<|user|>
මෙම නීතිමය ලේඛනය කියවා පහත ආකෘතියට අනුව පිළ


In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)


In [ ]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers import TrainerCallback

# ---- callback to print train + eval loss whenever logs happen ----
class PrintLossCallback(TrainerCallback):
    def __init__(self):
        self.last_train_loss = None
        self.last_eval_loss = None

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return

        # training loss appears in normal logging steps
        if "loss" in logs:
            self.last_train_loss = logs["loss"]

        # eval loss appears right after evaluation
        if "eval_loss" in logs:
            self.last_eval_loss = logs["eval_loss"]

        # print whenever we logged something (so you see both as soon as available)
        step = state.global_step
        tl = f"{self.last_train_loss:.4f}" if self.last_train_loss is not None else "N/A"
        vl = f"{self.last_eval_loss:.4f}" if self.last_eval_loss is not None else "N/A"
        print(f"[step {step}] train_loss={tl} | val_loss={vl}")

# ---- Ensure pad token exists ----
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

max_length = 1024  # reduce to 512 if OOM (also speeds up)

sft_args = SFTConfig(
    output_dir="/content/legal_sinhala_sft_out",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,

    # --- log + eval every 200 steps ---
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="steps",
    eval_steps=200,

    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,

    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",

    dataset_text_field="text",
    max_length=max_length,

    bf16=True,
    dataloader_num_workers=2,
)


trainer = SFTTrainer(
    model=model,
    args=sft_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    peft_config=lora_config,
    processing_class=tokenizer,   # TRL 0.27.1
)

trainer.add_callback(PrintLossCallback())
trainer.train()


warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss,Validation Loss
200,0.489812,0.345813
400,0.306723,0.283955
600,0.251143,0.255062
800,0.234427,0.249958


[step 200] train_loss=0.4898 | val_loss=N/A
[step 200] train_loss=0.4898 | val_loss=0.3458
[step 400] train_loss=0.3067 | val_loss=0.3458
[step 400] train_loss=0.3067 | val_loss=0.2840
[step 600] train_loss=0.2511 | val_loss=0.2840
[step 600] train_loss=0.2511 | val_loss=0.2551


[step 800] train_loss=0.2344 | val_loss=0.2551
[step 800] train_loss=0.2344 | val_loss=0.2500
[step 800] train_loss=0.2344 | val_loss=0.2500


TrainOutput(global_step=800, training_loss=0.32052614212036135, metrics={'train_runtime': 5076.8843, 'train_samples_per_second': 1.261, 'train_steps_per_second': 0.158, 'total_flos': 5.051739595780915e+16, 'train_loss': 0.32052614212036135})

In [ ]:
from datasets import Dataset

def tokenize_for_eval(examples):
    out = tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_length,
        padding=False,  # Trainer will pad dynamically if a data collator is used
    )
    # For causal LM eval, labels usually equal input_ids
    out["labels"] = out["input_ids"].copy()
    return out

test_tok = test_ds.map(tokenize_for_eval, batched=True, remove_columns=test_ds.column_names)

test_metrics = trainer.evaluate(eval_dataset=test_tok)
print(test_metrics)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

[step 800] train_loss=0.2344 | val_loss=0.2402
{'eval_loss': 0.24019497632980347, 'eval_runtime': 96.9882, 'eval_samples_per_second': 4.124, 'eval_steps_per_second': 4.124}


In [ ]:
save_dir = "/content/drive/MyDrive/FYP_models"
trainer.model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)

Saved to: /content/drive/MyDrive/FYP_models


In [ ]:
import torch

def generate(doc: str, max_new_tokens=350):
    prompt = (
        f"<|system|>\n{SYSTEM}\n"
        f"<|user|>\n"
        f"මෙම නීතිමය ලේඛනය කියවා පහත ආකෘතියට අනුව පිළිතුර ලබා දෙන්න:\n\n"
        f"1) සාරාංශය\n2) නීතිමය විශ්ලේෂණය\n3) ක්‍රියාමාර්ග/තීරණ උපදෙස්\n\n"
        f"ලේඛනය:\n{doc}\n"
        f"<|assistant|>\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_seq_length).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

# Example:
# print(generate("ඔබගේ සිංහල නීතිමය ලේඛන පෙළ මෙහි දාන්න..."))


In [ ]:
!pip install -q pypdf
!pip install -q peft accelerate


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_PATH = "/content/drive/MyDrive/FYP_models"  # folder containing adapter_model.safetensors

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer from adapter folder (important for chat template)
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH, trust_remote_code=True)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [ ]:
from google.colab import files
uploaded = files.upload()   # choose your PDF
pdf_path = next(iter(uploaded.keys()))
print("Uploaded:", pdf_path)


Saving 2022-12-30(I-I)S.pdf to 2022-12-30(I-I)S (1).pdf
Uploaded: 2022-12-30(I-I)S (1).pdf


In [ ]:
!apt-get install -y tesseract-ocr tesseract-ocr-sin
!pip install -q pypdf pdf2image pytesseract pillow
!apt-get update
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr tesseract-ocr-sin
!pip install -q pytesseract pdf2image pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-sin is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,316 kB]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelea

In [ ]:
import re
from pypdf import PdfReader
from pdf2image import convert_from_path
import pytesseract

def extract_text_normal(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    pages = []
    for page in reader.pages:
        pages.append(page.extract_text() or "")
    text = "\n".join(pages)

    text = text.replace("\x00", " ")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def extract_text_sinhala_ocr(pdf_path: str) -> str:
    images = convert_from_path(pdf_path, dpi=300)
    all_text = []
    for img in images:
        txt = pytesseract.image_to_string(img, lang="sin")
        all_text.append(txt)

    text = "\n".join(all_text)
    text = text.replace("\x00", " ")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def has_sinhala_unicode(text: str, min_chars: int = 30) -> bool:
    # Sinhala Unicode range: U+0D80–U+0DFF
    sinhala_chars = re.findall(r"[\u0D80-\u0DFF]", text)
    return len(sinhala_chars) >= min_chars

def looks_garbled(text: str) -> bool:
    # Heuristic: lots of weird symbols/carets and very low Sinhala Unicode presence
    weird = len(re.findall(r"[\^&%$#@_~=<>\\|]", text))
    letters = len(re.findall(r"[A-Za-z]", text))
    # If it is mostly Latin-ish garbage and symbols, treat as garbled
    return (weird > 30 and letters > 200 and not has_sinhala_unicode(text, 5))

def extract_text_auto(pdf_path: str) -> str:
    print("Trying normal PDF text extraction...")
    text = extract_text_normal(pdf_path)

    if len(text) < 150:
        print("Too little text -> using Sinhala OCR...")
        return extract_text_sinhala_ocr(pdf_path)

    if not has_sinhala_unicode(text) or looks_garbled(text):
        print("Text is not Sinhala Unicode / looks garbled -> using Sinhala OCR...")
        return extract_text_sinhala_ocr(pdf_path)

    print("Normal extraction produced Sinhala Unicode text.")
    return text


In [ ]:
pdf_text = extract_text_auto(pdf_path)

print("Characters:", len(pdf_text))
print(pdf_text[:800])


Trying normal PDF text extraction...
Text is not Sinhala Unicode / looks garbled -> using Sinhala OCR...
Characters: 66601
ලබා රුරා20255 සාරා තරයේ 50 ටු

අංක 2,313 - 2022 දෙසැමබර්‌ මස 30 වැනි සිකුරාදා - 2022.12.30

 

 

(රජයේ බලයපිට ප්‍රසිද්ධ කරන ලදී)

1 වැනි කොටස : 1 වැනි ඡෙදය - සාමාන්‍ය

(වෙන වෙනම ගොනු කර ගත හැකි පරිදි සැම කොටසකට ම අයත්‌ එක්‌ එක්‌ භාෂාවකට චෙන වෙනම පිටු අංක යොදා ඇත)

පිටුව පිටුව
රජයේ නිවේදන ... ... ... 1854
මිල පාලන දන්වීම්‌ ... ..- ... ~~~
ශ්‍රී ලංකා මහ බැංකුවේ දන්වීම ... ..- ... ~~~
ශ්‍රී ලංකාණ්ඩුවේ ගිණුම්‌ ප්‍රකාශන -.. ... -.. ~~~
ආදායම්‌ හා වියදම්‌ පිළිබඳ වාර්තා ... ..- ... ~~~
විවිධ දෙපාර්තමේන්තු දන්වීම්‌ -:- -.-- -:- 1859
නැවියන්ට දන්වීම්‌ -.. ..- -.. ~~~
සුරාබදු ආඥාපනත පිළිබඳ දන්වීම්‌ -.. ..- -.. ~~~

ජනාධිපතිතුමාණන්‌ විසින්‌ කරන ලද ප්‍රකාශන ආදිය -.. ~~~
ජනාධිපතිතුමාණන්‌ විසින්‌ කරන ලද පත්කිරීම්‌ ආදිය -.. ~~~
අමාත්‍ය මණ්ඩලය විසින්‌ කරන ලද පත්කිරීම්‌ ආදිය . 1846
රාජ්‍ය සේවා කොමිෂන්‌ සභ


In [ ]:
import torch

SYSTEM = (
    "ඔබ ශ්‍රී ලංකාවේ නීතිමය ලේඛන විශ්ලේෂක AI සහායකයෙක්. "
    "ඔබගේ කාර්යය: සිංහල නීතිමය ලේඛන සාරාංශ කරමින්, නීතිමය විශ්ලේෂණයක් සහ "
    "පරිශීලකයාට තීරණ ගැනීමට උපකාරී ක්‍රියාමාර්ග/උපදෙස් සපයන්න. "
    "නීතිමය උපදෙස් ලෙස නොව, සාමාන්‍ය තොරතුරු ලෙස ඉදිරිපත් කරන්න."
)

@torch.inference_mode()
def summarize_with_qwen_legal(pdf_text: str, max_new_tokens: int = 500) -> str:
    # ✅ Use the SAME template you trained with
    prompt = (
        f"<|system|>\n{SYSTEM}\n"
        f"<|user|>\n"
        f"මෙම නීතිමය ලේඛනය කියවා පහත ආකෘතියට අනුව පිළිතුර ලබා දෙන්න:\n\n"
        f"1) සාරාංශය\n"
        f"2) නීතිමය විශ්ලේෂණය\n"
        f"3) ක්‍රියාමාර්ග/තීරණ උපදෙස්\n\n"
        f"අතිරේකව: JSON ආකෘතියෙන් පිළිතුර දෙන්න (keys: plain_summary, applies_to, main_points, actions_for_non_expert).\n\n"
        f"ලේඛනය:\n{pdf_text}\n"
        f"<|assistant|>\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        repetition_penalty=1.05,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    gen_ids = out[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


summary = summarize_with_qwen_legal(pdf_text)
print(summary)


අධික්ෂ මණ්ඩලයේ අන්‍ය අම්‍ය ප්‍රදානය කරන ලද්‌ ණය අයකර
ගැනීමේ (විශේෂ විධිවිධාන) පනතේ 4 වන වගන්තිය යටතේ යුන්‌
මුද්‍රණය කරන ලද ණය අයකර ගැනීමේ (විශේෂ විධිවිධාන) පනතේ
4 වන වගන්තිය යටතේ යුන්‌ මුද්‍රණය කරන ලද අංක 25/2004 සහ
2003.12.04 දින දරන පිඹුරේ නිරූපිත ලොට්‌ 2 දින්‌ ලිංක්‌ මුද්‍රණයක්‌
ඇති යුත්‌ එක්‌ මෙක්‌ මායිම්‌ තුළ පිහිටි, මුල්‌ මුද්‍රණයක්‌.
උතුරට : මෙම ඉඩමේ
